## 5. Tool use(langchain)

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from google.adk.agents import SequentialAgent, LlmAgent

### 1. 첫번째 에이전트

In [3]:
generator = LlmAgent(
    name="DraftWriter",
    description="주어진 주제에 대한 초기 초안 콘텐츠를 생성합니다.",
    instruction="사용자의 주제에 대한 짧고 유익한 단락을 작성하세요.",
    output_key="draft_text"  # 출력은 이 상태 키에 저장됩니다.
)

### 2. 두번째 에이전트

In [4]:
reviewer = LlmAgent(
    name="FactChecker",
    description="주어진 텍스트의 사실 정확성을 검토하고 구조화된 비판을 제공합니다.",
    instruction="""
    귀하는 세심한 사실 확인자입니다.
    1. 상태 키 'draft_text'에 제공된 텍스트를 읽으세요.
    2. 모든 주장의 사실 정확성을 신중하게 검증하세요.
    3. 귀하의 최종 출력은 두 개의 키를 포함하는 사전이어야 합니다:
       - "status": "ACCURATE" 또는 "INACCURATE"인 문자열.
       - "reasoning": 구체적 문제(있는 경우)를 인용하여 귀하의 상태에 대한 명확한 설명을 제공하는 문자열.
    """,
    output_key="review_output"  # 구조화된 사전은 여기에 저장됩니다.
)

### 3. Sequential Agent 생성

In [5]:
# SequentialAgent는 생성자가 검토자보다 먼저 실행되도록 합니다.
review_pipeline = SequentialAgent(
    name="WriteAndReview_Pipeline",
    sub_agents=[generator, reviewer]
)

# 실행 흐름:
# 1. generator 실행 -> 단락을 state['draft_text']에 저장.
# 2. reviewer 실행 -> state['draft_text']를 읽고 사전 출력을 state['review_output']에 저장.

generator 에이전트는 주어진 주제에 대한 초기 초안 단락을 생성하도록 설계되었습니다. 
짧고 유익한 작품을 작성하도록 지시받고 출력을 draft_text 상태 키에 저장합니다. 
reviewer 에이전트는 generator가 생성한 텍스트의 사실 확인자로 작용합니다. 
draft_text에서 텍스트를 읽고 사실 정확성을 검증하도록 지시받습니다. 
reviewer의 출력은 두 개의 키를 가진 구조화된 사전입니다: status는 텍스트가 "ACCURATE"인지 "INACCURATE"인지 나타내고, reasoning은 상태에 대한 설명을 제공합니다. 

이 dictionary review_output 상태 키에 저장됩니다. 
WriteAndReview_Pipeline이라는 SequentialAgent가 두 에이전트의 실행 순서를 관리하도록 생성됩니다. 
생성자가 먼저 실행된 다음 검토자가 실행되게 합니다. 전체 실행 흐름은 생성자가 텍스트를 생성하여 상태에 저장하는 것입니다. 
그 후, 검토자가 이 텍스트를 상태에서 읽고 사실 확인을 수행하며 발견 사항(상태 및 추론)을 상태로 다시 저장합니다. 
이 파이프라인은 별도의 에이전트를 사용하여 콘텐츠 생성 및 검토의 구조화된 프로세스를 허용합니다.   
참고: ADK의 LoopAgent를 활용한 대안 구현도 사용할 수 있습니다.